In [1]:
from network import *
from inference import *
import numpy as np
from tools import *
from os import listdir

%load_ext autoreload
%autoreload 2

In [2]:
seeds = [697752728, 4190089612, 1176914559, 3077924848, 315917623, 2544020234, 1077758578, 4071300106, 534591752, 3553386411]
Ts = [5,10,30,50,100]
K_exps = [3,6,9]

N = 100
avg_degree_A0 = 100
K_aff = 3
dirichlet = 1
prob_ratio = 0.1
symmetric = True

In [3]:
in_folder = "../data/input/synth/"
out_folder = "../data/output/synth/"

In [4]:
params = np.load(in_folder + "params_3077924848_50_9.npz")
estimates = np.load(out_folder + "params_3077924848_50_9_Exp_697752728_None.npz")

mu, Z, u, w = params["mu"], params["Z"], params["u"], params["w"]
mu_est, Q, u_est, v_est, w_est = estimates["mu"], estimates["Q"], estimates["u"], estimates["v"], estimates["w"]

In [5]:
print("Accuracy predicting mu: ", evaluate_memberships(mu, mu_est, mu, mu_est)[0])
print("Accuracy predicting u: ", evaluate_memberships(u, u_est, mu, mu_est)[0])
print("Accuracy predicting Z: ", calculate_AUC(Z, Q))

Accuracy predicting mu:  0.4770159380140865
Accuracy predicting u:  0.9674491503251211
Accuracy predicting Z:  0.8329543409749067


In [6]:
params = np.load(in_folder + "params_315917623_5_3.npz")
estimates = np.load(out_folder + "params_315917623_5_3_NoExp_42_None.npz")

u, w = params["u"], params["w"]
u_est, v_est, w_est = estimates["u"], estimates["v"], estimates["w"]

In [7]:
print("Accuracy predicting u: ", evaluate_memberships(u, u_est, mu, mu_est)[0])

Accuracy predicting u:  0.7309476553638149
